In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)

# Explore public HUD REAC data

Author: Jack Vandeleuv

Data files:
* [Public Housing Physical Inspection Scores (2016-2021)](https://www.huduser.gov/portal/datasets/pis.html#2021_data-collapse)
* [Multifamily Physical Inspection Scores (2016-2021)](https://www.huduser.gov/portal/datasets/pis.html#2021_data-collapse)

This notebook will explore the publicly released scores from the U.S. Department of Housing and Urban Development's Real Estate Assessment Center (HUD REAC). These scores are derived from physical inspections on HUD properties.

The goal of this notebook is to aggregate REAC data on a city-by-city basis.

# Loading and cleaning

Place the data files in the data directory and load into pandas.

In [3]:
WORKING_DIRECTORY_IN = '../data/raw/reac/'
MULTIFAMILY_FILES = [
    'multifamily_physical_inspection_scores_0321.xlsx',
    'multifamily_physical_inspection_scores_0620.xlsx',
    'multifamily-physical-inspection-scores-2016.xlsx',
    'multifamily-physical-inspection-scores-2018.xlsx',
    'multifamily-physical-inspection-scores-2019.xlsx'
]

PUBLIC_HOUSING_FILES = [
    'public_housing_physical_inspection_scores_0321.xlsx',
    'public_housing_physical_inspection_scores_0620.xlsx',
    'public-housing-physical-inspection-scores-2016.xlsx',
    'public-housing-physical-inspection-scores-2018.xlsx',
    'public-housing-physical-inspection-scores-2019.xlsx'
]

The zipcode columns are inconsistently named, so we'll rename ZIPCODE to ZIP.

In [4]:
public_dfs = [
    pd.read_excel(WORKING_DIRECTORY_IN + file)
    .rename(columns={
        'ZIPCODE': 'ZIP',
        'DEVELOPMENT_ID': 'PLACE_ID',
        'DEVELOPMENT_NAME': 'PLACE_NAME',
        'ADDRESS ': 'ADDRESS'
        })
    .assign(INSPECTION_TYPE='PUBLIC')
    for file in PUBLIC_HOUSING_FILES
]

multi_dfs = [
    pd.read_excel(WORKING_DIRECTORY_IN + file)
    .rename(columns={
        'ZIPCODE': 'ZIP',
        'PROPERTY_ID': 'PLACE_ID',
        'PROPERTY_NAME': 'PLACE_NAME',
        })
    .assign(INSPECTION_TYPE='MULTIFAMILY')
    for file in MULTIFAMILY_FILES
]

We'll also remove PHA_CODE and PHA_NAME from the public housing dfs so the columns match with the multifamily dfs.

In [5]:
COLUMNS_TO_DROP = ['PHA_CODE', 'PHA_NAME']
for i, df in enumerate(public_dfs):
    public_dfs[i] = df.drop(columns=[
        col for col in COLUMNS_TO_DROP 
        if col in df.columns])

Now we'll combine all the dataframes together.

In [6]:
multi_dfs.extend(public_dfs)
df = pd.concat(multi_dfs, axis=0)

Drop duplicate inspection (any records that share an inspection id).

In [7]:
df = df.drop_duplicates(subset='INSPECTION_ID')

Let's check the null percentage in each column..

In [8]:
(df.isnull().mean() * 100).round(2).sort_values(ascending=False)

FIPS_STATE_CODE     81.44
STATE_NAME          18.56
CBSA_NAME            9.33
ADDRESS              1.92
CBSA_CODE            0.04
COUNTY_NAME          0.04
COUNTY_CODE          0.04
LATITUDE             0.04
LONGITUDE            0.04
ZIP                  0.03
LOCATION_QUALITY     0.03
PLACE_NAME           0.00
INSPECTION_SCORE     0.00
INSPECTION_TYPE      0.00
INSPECTION_DATE      0.00
STATE_CODE           0.00
PLACE_ID             0.00
CITY                 0.00
INSPECTION_ID        0.00
dtype: float64

City, state code, and inspection score don't have any null values, which is good. For now, we'll leave the other columns with nulls.

It looks like we have two different methods of specifying state.

In [9]:
df.sample(n=5, random_state=6).loc[:, ['STATE_NAME', 'STATE_CODE']]

,STATE_NAME,STATE_CODE
20332,AL,1
5737,KY,21
4047,NaN,IL
22709,TX,48
18484,NaN,MN


We also have two different state code formats. We'll combine these into a single column with only the two-letter state codes.

In [10]:
def combine_columns(row):
    if pd.isnull(row['STATE_NAME']):
        return row['STATE_CODE']
    else:
        return row['STATE_NAME']

df['STATE'] = df.apply(combine_columns, axis=1)
df = df.drop(columns=['STATE_NAME', 'STATE_CODE'])

Standardize the city names to be upper-case.

In [11]:
df.CITY = df.CITY.str.upper()

Convert the datetimes to a standard format.

In [12]:
df['INSPECTION_DATE'] = pd.to_datetime(df['INSPECTION_DATE'], 
                                       infer_datetime_format=True)

Let's look at this distribution of inspections by date range.

In [13]:
df.groupby(by=['INSPECTION_TYPE', df.INSPECTION_DATE.dt.year]).size()

INSPECTION_TYPE  INSPECTION_DATE
MULTIFAMILY      2013                2949
                 2014                8542
                 2015               10762
                 2016                9104
                 2017                7471
                 2018               11856
                 2019               11286
                 2020                1835
                 2021                   2
PUBLIC           2005                   1
                 2012                   5
                 2013                 336
                 2014                1739
                 2015                3272
                 2016                2424
                 2017                3302
                 2018                2438
                 2019                2566
                 2020                 879
dtype: int64

The bulk of inspections in the dataset occurred between 2013 and 2020, with a drop-off when the 2019 pandemic started. 

Drop the pre-2013 data and the post-2020 data, which is only a few inspections.

In [14]:
df = df[
    (df.INSPECTION_DATE.dt.year >= 2013) &
    (df.INSPECTION_DATE.dt.year <= 2020)
]   

Check the minimum and maximum length of city names as a simple validation.

In [15]:
print('Longest names:', df.loc[df.CITY.str.len() == df.CITY.str.len().max()].CITY.unique())
print('Shortest names', df.loc[df.CITY.str.len() == df.CITY.str.len().min()].CITY.unique())

Longest names: ['HOT SPRINGS NATIONAL PARK']
Shortest names ['ADA' 'IVA' 'LEE' 'ONA' 'DIX' 'AVA' 'RYE' 'OPP' 'OLA' 'ELY' 'BAY' 'RIO'
 'ROY' 'ORD' 'YOE' 'VAN' 'INA']


Check the range of inspection scores for validation.

In [16]:
print(
    'Min:', df.INSPECTION_SCORE.min(), 
    '\nMax:', df.INSPECTION_SCORE.max()
)

Min: 0 
Max: 100


There are 55 unique states represented, with some like GU representing territories.

In [17]:
print('Unique state names:', len(df.STATE.unique()))
df.STATE.unique()

Unique state names: 55


array(['IN', 'ID', 'CA', 'NY', 'MO', 'UT', 'TX', 'MA', 'NJ', 'NM', 'TN',
       'IL', 'CT', 'PA', 'NC', 'OH', 'AZ', 'IA', 'MI', 'RI', 'KY', 'MD',
       'GA', 'OR', 'SD', 'WY', 'FL', 'MT', 'ME', 'CO', 'MN', 'WA', 'VA',
       'WI', 'LA', 'AR', 'OK', 'KS', 'MS', 'NE', 'SC', 'PR', 'WV', 'NH',
       'NV', 'VI', 'DC', 'AL', 'VT', 'AK', 'ND', 'HI', 'DE', 'MP', 'GU'],
      dtype=object)

Export our cleaned data from 2013-2019. (Reserve the 2020 data as a validation set.)

In [18]:
reac_13_19 = df[df.INSPECTION_DATE.dt.year < 2019].copy(deep=True)

Combine our data at the city level for compatibility with our other datasets.

In [19]:
reac_13_19['CITYSTATE'] = reac_13_19.CITY.str.upper() + ',' + reac_13_19.STATE.str.upper()

In [20]:
reac_13_19 = reac_13_19 \
    .groupby(by='CITYSTATE') \
    .INSPECTION_SCORE \
    .mean() \
    .reset_index() \
    .rename(columns={'INSPECTION_SCORE': 'AVG_SCORE'})

In [21]:
WORKING_DIRECTORY_OUT = '../data/processed/'

reac_13_19.to_csv(
    WORKING_DIRECTORY_OUT + 'reac_13-19.csv', 
    sep=',', 
    index=False)